## Introduction
Hospital readmissions represent a significant challenge in the healthcare
sector, both as an indicator of care quality and a driver of escalating costs.
When a patient is re-admitted to the hospital within a short period after
discharge, it not only indicates potential gaps in care but also adds to the
financial burden on the healthcare system. In particular, readmissions of
diabetic patients have been noted to contribute significantly to these
costs. Therefore, being able to predict such readmissions can lead to
improved patient care and substantial cost savings.

## Dataset Description
* **encounter_id:** Unique identifier of the encounter.

* **country:** The country in which the encounter took place.

* **patient_id:** Identifier of the patient.

* **race:** Patient’s race.

* **gender:** Patient’s gender.

* **age:** Patient’s age bracket.

* **weight:** Patient’s weight.

* **payer_code:** Code of the health insurance provider (if there is one).

* **outpatient_visits_in_previous_year:** Number of outpatient visits (visits made with the intention of leaving on the same day) the patient made to the hospital in the year preceding the encounter.

* **emergency_visits_in_previous_year:** Number of emergency visits the patient made to the hospital in the year preceding the encounter.

* **inpatient_visits_in_previous_year:** Number of inpatient visits (visits with the intention to stay overnight) the patient made to the hospital in the year preceding the encounter.

* **admission_type:** Type of admission of the patient (e.g. Emergency, Urgent, etc.).

* **medical_specialty:** Medical specialty on which the patient was admitted.

* **average_pulse_bpm:** Average pulse of the patient during their stay in the hospital in beats per minute.

* **discharge_disposition:** Destination given to the patient after being discharged.

* **admission_source:** Source of the patient before being admitted in the current encounter.

* **length_of_stay_in_hospital:** Number of days between admission and discharge.

* **number_lab_tests:** Number of lab tests performed during the encounter.

* **non_lab_procedures:** Number of non-lab procedures performed during the encounter.

* **number_of_medications:** Number of distinct types of medication administered during the encounter.

* **primary_diagnosis:** Primary diagnosis (coded as the first three digits of ICD9).

* **secondary_diagnosis:** Secondary diagnosis (first three digits of ICD9).

* **additional_diagnosis:** Additional secondary diagnosis (first three digits of ICD9).

* **number_diagnoses:** Number of diagnoses entered into the system.

* **glucose_test_result:** Range of the glucose test results or if the test was not taken. Values: “>200,” “>300,” “normal,” and “none” if not measured.

* **a1c_test_result:** Range of the A1C test results or if the test was not taken. Values: “>8” if greater than 8%, “>7” if greater than 7% but less than 8%, “normal” if less than 7%, and “none” if not measured.

* **change_in_meds_during_hospitalization:** Indicates if there was a change in diabetic medications (dosage or generic name). Values: “change” and “no change”.

* **prescribed_diabetes_meds:** Yes if the patient has diabetes medication prescribed. No otherwise.

* **medication:** List containing all generic names for the medications prescribed to the patient during the encounter. An empty list if no medication was prescribed.

* **readmitted_binary:** Binary target: Yes if the patient was readmitted in less than 30 days, No otherwise.

* **readmitted_multiclass:** Multiclass target: “<30 days” if the patient was readmitted in less than 30 days after being discharged. “>30 days” if the patient was readmitted to the hospital but only after more than 30 days after the current discharge. No otherwise.

**Main decisions to take:** \
Duplicated patient_id \
Missing values \
Outliers \
High cardinality \
Feature engineering \
Feature selection

Everything used just for visualization is commented

## Data Exploration and Preprocessing

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="whitegrid")
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### Data loading

In [5]:
data = pd.read_csv("files/train.csv")
pd.set_option('display.max_columns', None)

### Data Inspection

In [6]:
#data.head()

In [7]:
#data.info()

As we observe the data, we can see that we have a strange value ('?') in a lot of columns. Let's take a closer look

In [8]:
#for col in data.columns:
   # print(col + ':', data[col].unique(), '\n')

In [9]:
#for col in data.columns:
 #   print(col + ':', len(data[col].unique()), '\n')

We can see by the unique values of each column that we have some problems in our dataset:
* We will set the index as **encounter_id** as it is the unique identifier
* **country** only has one value, so it has no influence to our future predictions. We should drop it
* **patient_id** has duplicated values. We will do some feature engineering later based on this
* **gender** has a value 'Unknown/Invalid'. We will assume it means the data was missing
* **age** and **weight** are categorical variables. We will get to this later 
* **admission_type** and some other categories have values like 'Not Available' and 'Not Mapped'. We will assume it means the data was missing
* **admission_source** has strings with trailing blank spaces
* **medical_specialty**, **discharge_disposition**, **admission_source**, **primary_diagnosis**, **secondary_diagnosis**, **additional_diagnosis** and **medication** seem to have a very high number of categories. We will get to this later

Set encounter_id as index

In [10]:
data = data.set_index('encounter_id')

Let's visualize our target variable

In [11]:
# #fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

# # Countplot with value counts 
# sns.countplot(x="readmitted_binary", data=data, ax=axes[0])
# axes[0].set_title("Distribution of Target Values")
# for p in axes[0].patches:
#     axes[0].annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', xytext=(0, 1), textcoords='offset points')

# # Pie chart
# data.readmitted_binary.value_counts().plot.pie(autopct="%.1f%%", ax=axes[1])
# axes[1].set_title("Proportion of Target Value")
# plt.tight_layout()
# plt.show()


As we can see, we are dealing with an imbalanced dataset. We will get to this later in our modeling. For now we will just encode the target variable as numeric so it can be used later in the model

In [12]:
data['readmitted_binary'] = data['readmitted_binary'].replace({'No': 0, 'Yes': 1})

Drop country

In [13]:
data.drop(['country'], inplace=True, axis=1)

Remove trailing blank spaces

In [14]:
categorical_columns = data.select_dtypes(include=['object']).columns
data[categorical_columns] = data[categorical_columns].apply(lambda x: x.str.lstrip())

Replace 'Not Available', 'Not Mapped', '?', 'Unknown/Invalid' with null values

In [15]:
data = data.replace(['Not Available', 'Not Mapped', '?', 'Unknown/Invalid'], np.nan)

So now that we have fixed some of the problems in our data, we can check the missing values

In [16]:
# # Calculate the number of missing values per column
# missing_count = data.isnull().sum()

# # Calculate the percentage of missing values per column
# missing_percentage = (missing_count / len(data) * 100).round(2)

# # Create a DataFrame to display the results
# missing_info = pd.DataFrame({
#     'Missing Count': missing_count,
#     'Missing Percentage': missing_percentage
# })
# # 
# # Print the missing information
# columns_with_missing = missing_info[missing_info['Missing Count'] > 0]
# columns_with_missing

Let's start handling the missing values

**Race**

We have about 7% of the data missing, so we can use mode imputation or fill with a placeholder value 'Unknown'. We decided to fill with 'Unknown'

In [17]:
data['race'].fillna('Unknown', inplace=True)

**Gender**

We only have 3 missing values, so we can drop these rows as it won't cause a big loss of information, or we can impute with the mode. We chose to impute with the mode as we don't want to have problems in the future when dealing with the test set

In [18]:
data['gender'].fillna(data['gender'].mode()[0], inplace=True)

**Age**

Let's turn the 'age' column into numeric by taking the midpoint of each categorie

In [19]:
# Define a mapping from age groups to numeric values
age_mapping = {
    '[0-10)': 5,
    '[10-20)': 15,
    '[20-30)': 25,
    '[30-40)': 35,
    '[40-50)': 45,
    '[50-60)': 55,
    '[60-70)': 65,
    '[70-80)': 75,
    '[80-90)': 85,
    '[90-100)': 95
}

# map values to numerical values
data['age'] = data['age'].replace(age_mapping)

Now, we will check the distribution to see what imputation method is suitable

In [20]:
# from scipy.stats import kstest

# sns.histplot(data['age'], kde=True) 
# plt.title('Histogram and KDE Plot')
# plt.show()

# # Perform the Kolmogorov-Smirnov test
# stat, p = kstest(data['age'], 'norm')

# # Check the p-value to determine normality
# if p > 0.05:
#     print("Data appears to be normally distributed")
# else:
#     print("Data does not appear to be normally distributed")

Based on the information we got the data does not appear to be normally distributed, so a suitable approach would be to impute the missing values with the median or use KNN. We chose to use the median.

In [21]:
data['age'] = data['age'].fillna(data['age'].median())

**Weight**

We have a very high percentage of missing values (97%). We can fill the missing values with a placeholder value or we can drop it. We decided to drop it

In [22]:
data.drop(columns=['weight'], inplace=True)

**Payer_code**

Fill with 'Unknown' as it has a very high percentage of missing values (40%)

In [23]:
data['payer_code'].fillna('Unknown', inplace=True)

**Glucose_test_result** and **a1c_test_result**

The missing values on both of these columns are not really missing values, they mean that the test was not taken, so we will fill the missing values accordingly

In [24]:
data['glucose_test_result'].fillna('Not Taken', inplace=True)
data['a1c_test_result'].fillna('Not Taken', inplace=True)

**Admission_type**

Fill with 'Unknown' as it is a suitable approach based on the % of missing values

In [25]:
data['admission_type'].fillna('Unknown', inplace=True)

**Medical_specialty**

Fill with 'Unknown' as it has a very high percentage of missing values (49%)

In [26]:
data['medical_specialty'].fillna('Unknown', inplace=True)

**Discharge_disposition**

Fill with 'Unknown' as it is a suitable approach based on the % of missing values

In [27]:
data['discharge_disposition'].fillna('Unknown', inplace=True)

**Admission_source**

Fill with 'Unknown' as it is a suitable approach based on the % of missing values

In [28]:
data['admission_source'].fillna('Unknown', inplace=True)

**Primary_diagnosis**

Fill with 'Unknown' as it is a suitable approach based on the % of missing values

In [29]:
data['primary_diagnosis'].fillna('Unknown', inplace=True)

**Secondary_diagnosis**

Fill with 'Unknown' as it is a suitable approach based on the % of missing values

In [30]:
data['secondary_diagnosis'].fillna('Unknown', inplace=True)

**Additional_diagnosis**

Fill with 'Unknown' as it is a suitable approach based on the % of missing values

In [31]:
data['additional_diagnosis'].fillna('Unknown', inplace=True)

Let's check if all the changes were applied and there are no missing values

In [32]:
#data.isna().sum()

Now we will do some data visualization along with trying to reduce the cardinality of the categorical features, in order to reduce the dimensionality. We will also do some feature engineering. Let's check the number of categories in each categorical variable

In [33]:
# categorical_columns = data.select_dtypes(include=['object', 'category']).columns

# # Create an empty list to store DataFrames for each column
# dfs = []

# # Iterate through each categorical column
# for column in categorical_columns:
#     num_categories = data[column].nunique()
#     df_temp = pd.DataFrame({'Number_of_Categories': [num_categories]}, index=[column])
#     dfs.append(df_temp)

# # Concatenate the DataFrames into the final result
# categories_df = pd.concat(dfs)

# # Display the resulting DataFrame
# categories_df


**Race**

In [34]:
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))


# # Countplot with value counts 
# sns.countplot(x="race", data=data, ax=axes[0])
# axes[0].set_title("Distribution of Race")
# for p in axes[0].patches:
#     axes[0].annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', xytext=(0, 1), textcoords='offset points')

# # Pie chart
# data['race'].value_counts().plot.pie(autopct="%.1f%%", ax=axes[1])
# axes[1].set_title("Proportion of Race")
# plt.tight_layout()
# plt.show()

In [35]:
# sns.catplot(x = 'race', y = 'readmitted_binary', data=data, kind = "bar", height= 5, aspect=2)
# plt.title("Readmitted Probability")
# plt.show()

**Gender**

In [36]:
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

# # Countplot with value counts 
# sns.countplot(x="gender", data=data, ax=axes[0])
# axes[0].set_title("Distribution of Gender")
# for p in axes[0].patches:
#     axes[0].annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', xytext=(0, 1), textcoords='offset points')

# # Pie chart
# data['gender'].value_counts().plot.pie(autopct="%.1f%%", ax=axes[1])
# axes[1].set_title("Proportion of Gender")
# plt.tight_layout()
# plt.show()


In [37]:
# sns.catplot(x = 'gender', y = 'readmitted_binary', data=data, kind = "bar", height= 5)
# plt.title("Readmitted Probability")
# plt.show()

**Payer_code**

In [38]:
# plt.figure(figsize=(20, 6))

# # Create a countplot
# ax = sns.countplot(x="payer_code", data=data)

# # Set labels and title
# plt.title("Distribution of Payer_code")
# plt.xlabel("Payer code")
# plt.ylabel("Count")

# # Rotate x-axis labels
# ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')

# # Annotate each bar with its count
# for p in ax.patches:
#     ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
#                 ha='center', va='bottom', xytext=(0, 5), textcoords='offset points')

# plt.show()

In [39]:
# # Grouped data
# grouped_data = data.groupby('payer_code')['readmitted_binary'].mean()

# # Calculate value counts for each category
# value_counts = data['payer_code'].value_counts()

# # Sort specialties based on mean values
# sorted_data = grouped_data.sort_values(ascending=False)

# # Set the figure size using sns
# plt.figure(figsize=(20, 6))

# # Plot the results with sns
# ax = sns.barplot(x=sorted_data.index, y=sorted_data.values)

# # Annotate with value counts on top of each bar
# for p, label in zip(ax.patches, value_counts.loc[sorted_data.index]):
#     ax.annotate(f'{label}', (p.get_x() + p.get_width() / 2., p.get_height()),
#                 ha='center', va='center', xytext=(0, 10), textcoords='offset points')

# plt.title('Probability of Readmission by payer_code with respective value counts')
# plt.ylabel('Probability of Readmission')
# plt.xlabel('payer_code')
# plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
# plt.show()


There is no pattern we can see on the data so we will just create a binary column that says if the patient has insurance or not. We will assume the null values we filled with 'Unknown' mean that the patient doesn't have insurance.

In [40]:
data['has_insurance'] = data['payer_code'].map(lambda x: 'No' if x == 'Unknown' else 'Yes')
data.drop(columns=['payer_code'], inplace=True)

In [41]:
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

# # Countplot with value counts 
# sns.countplot(x="has_insurance", data=data, ax=axes[0])
# axes[0].set_title("Distribution of Has_insurance")
# for p in axes[0].patches:
#     axes[0].annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', xytext=(0, 1), textcoords='offset points')

# # Pie chart
# data['has_insurance'].value_counts().plot.pie(autopct="%.1f%%", ax=axes[1])
# axes[1].set_title("Proportion of Has_insurance")
# plt.tight_layout()
# plt.show()


In [42]:
# sns.catplot(x = 'has_insurance', y = 'readmitted_binary', data=data, kind = "bar", height= 5)
# plt.title("Readmitted Probability")
# plt.show()

**Admission_type**

In [43]:
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

# # Countplot with value counts 
# sns.countplot(x="admission_type", data=data, ax=axes[0])
# axes[0].set_title("Distribution of Admission_type")
# for p in axes[0].patches:
#     axes[0].annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', xytext=(0, 1), textcoords='offset points')

# # Pie chart
# data['admission_type'].value_counts().plot.pie(autopct="%.1f%%", ax=axes[1])
# axes[1].set_title("Proportion of Admission_type")
# plt.tight_layout()
# plt.show()


In [44]:
# sns.catplot(x = 'admission_type', y = 'readmitted_binary', data=data, kind = "bar", height= 5, aspect=2)
# plt.title("Readmitted Probability")
# plt.show()

Newborn and Trauma Center categories have very low prevalence in the dataset. We will join 'Trauma Center' with 'Emergency' and 'Newborn' with 'Urgent' because these share similar characteristics in a medical context.

In [45]:
mapped_admission_type = {"Trauma Center":"Emergency","Newborn":"Urgent"}
data['admission_type'] = data['admission_type'].replace(mapped_admission_type)

In [46]:
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

# # Countplot with value counts 
# sns.countplot(x="admission_type", data=data, ax=axes[0])
# axes[0].set_title("Distribution of Admission_type")
# for p in axes[0].patches:
#     axes[0].annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', xytext=(0, 1), textcoords='offset points')

# # Pie chart
# data['admission_type'].value_counts().plot.pie(autopct="%.1f%%", ax=axes[1])
# axes[1].set_title("Proportion of Admission_type")
# plt.tight_layout()
# plt.show()


In [47]:
# sns.catplot(x = 'admission_type', y = 'readmitted_binary', data=data, kind = "bar", height= 5)
# plt.title("Readmitted Probability")
# plt.show()

**Medical_specialty**

In [48]:
# plt.figure(figsize=(20, 6))

# # Create a countplot
# ax = sns.countplot(x="medical_specialty", data=data)

# # Set labels and title
# plt.title("Distribution of Medical_specialty")
# plt.xlabel("Medical Specialty")
# plt.ylabel("Count")

# # Rotate x-axis labels
# ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')

# # Annotate each bar with its count
# for p in ax.patches:
#     ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
#                 ha='center', va='bottom', xytext=(0, 5), textcoords='offset points')

# plt.show()


In [49]:
# # Grouped data
# grouped_data = data.groupby('medical_specialty')['readmitted_binary'].mean()

# # Calculate value counts for each category
# value_counts = data['medical_specialty'].value_counts()

# # Sort specialties based on mean values
# sorted_data = grouped_data.sort_values(ascending=False)

# # Set the figure size using sns
# plt.figure(figsize=(20, 6))

# # Plot the results with sns
# ax = sns.barplot(x=sorted_data.index, y=sorted_data.values)

# # Annotate with value counts on top of each bar
# for p, label in zip(ax.patches, value_counts.loc[sorted_data.index]):
#     ax.annotate(f'{label}', (p.get_x() + p.get_width() / 2., p.get_height()),
#                 ha='center', va='center', xytext=(0, 10), textcoords='offset points')

# plt.title('Probability of Readmission by Medical Specialty with respective value counts')
# plt.ylabel('Probability of Readmission')
# plt.xlabel('Medical Specialty')
# plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
# plt.show()


As we can see we have a lot of categories. For now we will keep them as is, but we can consider doing some grouping later

**Discharge_disposition**

In [50]:
# plt.figure(figsize=(20, 6))

# # Create a countplot
# ax = sns.countplot(x="discharge_disposition", data=data)

# # Set labels and title
# plt.title("Distribution of Discharge_disposition")
# plt.xlabel("Discharge disposition")
# plt.ylabel("Count")

# # Rotate x-axis labels
# ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')

# # Annotate each bar with its count
# for p in ax.patches:
#     ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
#                 ha='center', va='bottom', xytext=(0, 5), textcoords='offset points')

# plt.show()


In [51]:
# # Grouped data
# grouped_data = data.groupby('discharge_disposition')['readmitted_binary'].mean()

# # Calculate value counts for each category
# value_counts = data['discharge_disposition'].value_counts()

# # Sort specialties based on mean values
# sorted_data = grouped_data.sort_values(ascending=False)

# # Set the figure size using sns
# plt.figure(figsize=(20, 6))

# # Plot the results with sns
# ax = sns.barplot(x=sorted_data.index, y=sorted_data.values)

# # Annotate with value counts on top of each bar
# for p, label in zip(ax.patches, value_counts.loc[sorted_data.index]):
#     ax.annotate(f'{label}', (p.get_x() + p.get_width() / 2., p.get_height()),
#                 ha='center', va='center', xytext=(0, 10), textcoords='offset points')

# plt.title('Probability of Readmission by discharge_disposition with respective value counts')
# plt.ylabel('Probability of Readmission')
# plt.xlabel('discharge_disposition')
# plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
# plt.show()


Everything that contains the word Expired means that the patient has died, so he will not be readmitted. Let's join them accordingly

In [52]:
data.loc[data['discharge_disposition'].str.contains("expired", case=False, na=False), 'discharge_disposition'] =  "Expired"

Here we have a lot of categories. For now we will keep them all because the majority of them seem important

**Admission_source**

In [53]:
# plt.figure(figsize=(20, 6))

# # Create a countplot
# ax = sns.countplot(x="admission_source", data=data)

# # Set labels and title
# plt.title("Distribution of Admission_source")
# plt.xlabel("Admission source")
# plt.ylabel("Count")

# # Rotate x-axis labels
# ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')

# # Annotate each bar with its count
# for p in ax.patches:
#     ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
#                 ha='center', va='bottom', xytext=(0, 5), textcoords='offset points')

# plt.show()


In [54]:
# # Grouped data
# grouped_data = data.groupby('admission_source')['readmitted_binary'].mean()

# # Calculate value counts for each category
# value_counts = data['admission_source'].value_counts()

# # Sort specialties based on mean values
# sorted_data = grouped_data.sort_values(ascending=False)

# # Set the figure size using sns
# plt.figure(figsize=(20, 6))

# # Plot the results with sns
# ax = sns.barplot(x=sorted_data.index, y=sorted_data.values)

# # Annotate with value counts on top of each bar
# for p, label in zip(ax.patches, value_counts.loc[sorted_data.index]):
#     ax.annotate(f'{label}', (p.get_x() + p.get_width() / 2., p.get_height()),
#                 ha='center', va='center', xytext=(0, 10), textcoords='offset points')

# plt.title('Probability of Readmission by admission_source with respective value counts')
# plt.ylabel('Probability of Readmission')
# plt.xlabel('admission_source')
# plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
# plt.show()


We can see we have a lot of categories. To solve this we can try grouping the categories into fewer categories by using the following mapping:
* Keep 'Emergency Room'
* Group similar ones, like 'Transfer' and 'Referral'
* Join the rest into an 'Other' categorie


In [55]:
admission_source_mapping = {
    'Clinic Referral': 'Referral',
    'Physician Referral': 'Referral',
    'HMO Referral': 'Referral',
    'Transfer from another health care facility': 'Transfer',
    'Transfer from a hospital': 'Transfer',
    'Transfer from a Skilled Nursing Facility (SNF)': 'Transfer',
    'Transfer from hospital inpt/same fac reslt in a sep claim': 'Transfer',
    'Transfer from critial access hospital': 'Transfer',
    'Transfer from Ambulatory Surgery Center': 'Transfer',
    'Court/Law Enforcement': 'Other',
    'Extramural Birth': 'Other',
    'Normal Delivery': 'Other',
    'Sick Baby': 'Other',
    'Unknown': 'Other'
}

data['admission_source'] = data['admission_source'].replace(admission_source_mapping)

In [56]:
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

# # Countplot with value counts 
# sns.countplot(x="admission_source", data=data, ax=axes[0])
# axes[0].set_title("Distribution of Admission_source")
# for p in axes[0].patches:
#     axes[0].annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', xytext=(0, 1), textcoords='offset points')

# # Pie chart
# data['admission_source'].value_counts().plot.pie(autopct="%.1f%%", ax=axes[1])
# axes[1].set_title("Proportion of Admission_source")
# plt.tight_layout()
# plt.show()


In [57]:
# sns.catplot(x = 'admission_source', y = 'readmitted_binary', data=data, kind = "bar", height= 5, aspect=2)
# plt.title("Readmitted Probability")
# plt.show()

**Primary, secondary and additional diagnosis**

Before doing any kind of visualization we have to reduce the number of categories, as these 3 variables have really high cardinality. For this, we will use the mapping based on the ICD9 Codes

In [58]:
icd9_mapping = {
    '001-139': 'Infectious and parasitic diseases',
    '140-239': 'Neoplasms',
    '240-279': 'Endocrine, nutritional and metabolic diseases, and immunity disorders',
    '280-289': 'Diseases of the blood and blood-forming organs',
    '290-319': 'Mental disorders',
    '320-389': 'Diseases of the nervous system and sense organs',
    '390-459': 'Diseases of the circulatory system',
    '460-519': 'Diseases of the respiratory system',
    '520-579': 'Diseases of the digestive system',
    '580-629': 'Diseases of the genitourinary system',
    '630-679': 'Complications of pregnancy, childbirth, and the puerperium',
    '680-709': 'Diseases of the skin and subcutaneous tissue',
    '710-739': 'Diseases of the musculoskeletal system and connective tissue',
    '740-759': 'Congenital anomalies',
    '760-779': 'Certain conditions originating in the perinatal period',
    '780-799': 'Symptoms, signs, and ill-defined conditions',
    '800-999': 'Injury and poisoning',
}


def is_numeric(input_string):
    try:
        float(input_string)  # Try to convert the string to a float
        return True
    except ValueError:
        return False


def map_icd9(value):
    for key in icd9_mapping:
        code_range = key.split('-')
        if is_numeric(value):   
            if int(code_range[0]) <= float(value) <= int(code_range[1]):
                return icd9_mapping[key]
        else:
            if value.startswith(('E', 'V')):
                return 'External causes of injury and supplemental classification'
            else:
                return 'Unknown_diagnosis'


# Apply the mapping primary_diagnosis, secondary_diagnosis and additional_diagnosis 
data['primary_diagnosis'] = data['primary_diagnosis'].apply(map_icd9)
data['secondary_diagnosis'] = data['secondary_diagnosis'].apply(map_icd9)
data['additional_diagnosis'] = data['additional_diagnosis'].apply(map_icd9)

In [59]:
# for col in ["primary_diagnosis","secondary_diagnosis","additional_diagnosis"]:
#     plt.figure(figsize=(20, 6))

#     # Create a countplot
#     ax = sns.countplot(x=col, data=data)

#     # Set labels and title
#     plt.title(f"Distribution of {col}")
#     plt.xlabel(col)
#     plt.ylabel("Count")

#     # Rotate x-axis labels
#     ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')

#     # Annotate each bar with its count
#     for p in ax.patches:
#         ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
#                     ha='center', va='bottom', xytext=(0, 5), textcoords='offset points')

#     plt.show()

In [60]:
# for col in ["primary_diagnosis","secondary_diagnosis","additional_diagnosis"]:
#     sns.catplot(x = col, y = 'readmitted_binary', data=data, kind = "bar", height=6, aspect=3)
#     plt.title("Readmitted Probability")
#     plt.xticks(rotation=90)
#     plt.show()

**Glucose_test_result**

In [61]:
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

# # Countplot with value counts 
# sns.countplot(x="glucose_test_result", data=data, ax=axes[0])
# axes[0].set_title("Distribution of Glucose_test_result")
# for p in axes[0].patches:
#     axes[0].annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', xytext=(0, 1), textcoords='offset points')

# # Pie chart
# data['glucose_test_result'].value_counts().plot.pie(autopct="%.1f%%", ax=axes[1])
# axes[1].set_title("Proportion of Glucose_test_result")
# plt.tight_layout()
# plt.show()

In [62]:
# sns.catplot(x = 'glucose_test_result', y = 'readmitted_binary', data=data, kind = "bar", height= 5)
# plt.title("Readmitted Probability")
# plt.show()

We will turn this into numerical with the following mapping

In [63]:
glucose_mapping = {'>300': 3,
               '>200': 2,
               'Norm': 1,
               'Not Taken':0}

data['glucose_test_result'] = data['glucose_test_result'] .replace(glucose_mapping)

We will also create a new binary column if the patient has or hasn't taken the test

In [64]:
glucose_test_mapping = {3: 'Yes',
               2: 'Yes',
               1: 'Yes',
               0: 'No'}

data['glucose_test_taken'] = data['glucose_test_result'].replace(glucose_test_mapping)

In [65]:
# sns.catplot(x = 'glucose_test_taken', y = 'readmitted_binary', data=data, kind = "bar", height= 5)
# plt.title("Readmitted Probability")
# plt.show()

**A1c_test_result**

In [66]:
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

# # Countplot with value counts 
# sns.countplot(x="a1c_test_result", data=data, ax=axes[0])
# axes[0].set_title("Distribution of A1c_test_result")
# for p in axes[0].patches:
#     axes[0].annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', xytext=(0, 1), textcoords='offset points')

# # Pie chart
# data['a1c_test_result'].value_counts().plot.pie(autopct="%.1f%%", ax=axes[1])
# axes[1].set_title("Proportion of A1c_test_result")
# plt.tight_layout()
# plt.show()

In [67]:
# sns.catplot(x = 'a1c_test_result', y = 'readmitted_binary', data=data, kind = "bar", height= 5)
# plt.title("Readmitted Probability")
# plt.show()

We can see some strange information in this catplot. If the a1c is >8 it is considered suboptimal and associated with an higher risk of diabetes-related complications, so we would assume the patients in this range would be more likely to be readmitted, and the same would be true when the a1c is >7, but with a lower risk of diabetes-related complications. As we can see by the catplot the patients in this ranges don't have more probabilities of being readmitted. The patients in the 'Norm' , '>8' and '>7' ranges all have about the same probability of being readmitted (~10%) while the patients who didn't take the test are a little more likely to be readmitted (~11%). Based on this information we will create a new binary variable that represents if the patient has taken the test or not, and turn the original into numerical format as we did with glucose

In [68]:
a1c_mapping = {'>7': 2,
               '>8': 3,
               'Norm': 1,
               'Not Taken': 0}

data['a1c_test_result'] = data['a1c_test_result'] .replace(a1c_mapping)

In [69]:
a1c_test_mapping = {3: 'Yes',
               2: 'Yes',
               1: 'Yes',
               0: 'No'}

data['a1c_test_taken'] = data['a1c_test_result'] .replace(a1c_test_mapping)

In [70]:
# sns.catplot(x = 'a1c_test_taken', y = 'readmitted_binary', data=data, kind = "bar", height= 5)
# plt.title("Readmitted Probability")
# plt.show()

**Change_in_meds_during_hospitalization**

In [71]:
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

# # Countplot with value counts 
# sns.countplot(x="change_in_meds_during_hospitalization", data=data, ax=axes[0])
# axes[0].set_title("Distribution of Change_in_meds_during_hospitalization")
# for p in axes[0].patches:
#     axes[0].annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', xytext=(0, 1), textcoords='offset points')

# # Pie chart
# data['change_in_meds_during_hospitalization'].value_counts().plot.pie(autopct="%.1f%%", ax=axes[1])
# axes[1].set_title("Proportion of Change_in_meds_during_hospitalization")
# plt.tight_layout()
# plt.show()

In [72]:
# sns.catplot(x = 'change_in_meds_during_hospitalization', y = 'readmitted_binary', data=data, kind = "bar", height= 5)
# plt.title("Readmitted Probability")
# plt.show()

**Prescribed_diabetes_meds**

In [73]:
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

# # Countplot with value counts 
# sns.countplot(x="prescribed_diabetes_meds", data=data, ax=axes[0])
# axes[0].set_title("Distribution of Prescribed_diabetes_meds")
# for p in axes[0].patches:
#     axes[0].annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', xytext=(0, 1), textcoords='offset points')

# # Pie chart
# data['prescribed_diabetes_meds'].value_counts().plot.pie(autopct="%.1f%%", ax=axes[1])
# axes[1].set_title("Proportion of Prescribed_diabetes_meds")
# plt.tight_layout()
# plt.show()

In [74]:
# sns.catplot(x = 'prescribed_diabetes_meds', y = 'readmitted_binary', data=data, kind = "bar", height= 5)
# plt.title("Readmitted Probability")
# plt.show()

**Medication**

Before doing any kind of visualization we have to reduce the number of categories, as these variable has really high cardinality. To do this we decided to create binary columns for each medication to represent if the medication was or wasn't prescribed. We will also create the following columns:
* **num_prescribed_meds** - number of medications prescribed
* **prescribed_meds** - if medication was prescribed or not

In [75]:
import ast

data['medication'] = data['medication'].apply(ast.literal_eval)

# Extract unique medications from the lists
unique_medications = set(med for meds in data['medication'] for med in meds)

# Create binary columns for each unique medication type
for med in unique_medications:
    data[med] = data['medication'].apply(lambda x: 'Yes' if med in x else 'No')

data['num_prescribed_meds'] = data['medication'].apply(len)
data['prescribed_meds'] = data['num_prescribed_meds'].apply(lambda x: 'Yes' if x > 0 else 'No')

# drop medication column
data.drop(columns=['medication'], inplace=True)

In [76]:
# import math

# # Calculate the number of rows needed
# num_rows = math.ceil(len(unique_medications) / 3)

# # Create subplots with the calculated number of rows and 3 columns
# fig, axes = plt.subplots(nrows=num_rows, ncols=3, figsize=(15, 5 * num_rows))

# # Flatten the axes array to simplify indexing
# axes = axes.flatten()

# # Iterate over the columns and create pie charts
# for i, column in enumerate(unique_medications):
#     data[column].value_counts().plot(kind='pie', autopct='%1.1f%%', ax=axes[i])
#     axes[i].set_title(f'Pie Chart for {column}')

# # Adjust layout
# plt.tight_layout()

# # Show the plots
# plt.show()


We have a lot of columns where one of the classes has very low prevalence. We will drop all the columns where one of the classes has less than 1% prevalence

In [77]:
columns_to_drop =  []

for med in unique_medications:
    class_1_percentage = ((data[med].value_counts(normalize=True)*100)[1])
    if class_1_percentage < 1:
        columns_to_drop.append(med)

data.drop(columns=columns_to_drop, inplace=True)

In [78]:
# new_meds = [x for x in unique_medications if x not in columns_to_drop]

# # Calculate the number of rows needed
# num_rows = math.ceil(len(new_meds) / 3)

# # Create subplots with the calculated number of rows and 3 columns
# fig, axes = plt.subplots(nrows=num_rows, ncols=3, figsize=(15, 5 * num_rows))

# # Flatten the axes array to simplify indexing
# axes = axes.flatten()

# # Iterate over the columns and create pie charts
# for i, column in enumerate(new_meds):
#     data[column].value_counts().plot(kind='pie', autopct='%1.1f%%', ax=axes[i])
#     axes[i].set_title(f'Pie Chart for {column}')

# # Adjust layout
# plt.tight_layout()

# # Show the plots
# plt.show()

In [79]:
# # Filter out columns to drop
# new_meds = [x for x in unique_medications if x not in columns_to_drop]

# # Calculate the number of rows needed
# num_rows = math.ceil(len(new_meds) / 3)

# # Create subplots with the calculated number of rows and 3 columns
# fig, axes = plt.subplots(nrows=num_rows, ncols=3, figsize=(15, 5 * num_rows))

# # Flatten the axes array to simplify indexing
# axes = axes.flatten()

# # Iterate over the columns and create probability plots
# for i, column in enumerate(new_meds):
#     sns.barplot(x=column, y='readmitted_binary', data=data, ax=axes[i], ci=None, order=data[column].value_counts().index)
#     axes[i].set_title(f'Probability of readmission for {column}')

#     # Calculate and display percentages on each bar
#     total = len(data[column])
#     for p in axes[i].patches:
#         percentage = '{:.1f}%'.format(100 * p.get_height())
#         x = p.get_x() + p.get_width() / 2
#         y = p.get_height()
#         axes[i].annotate(percentage, (x, y), ha='center', va='bottom')

# # Adjust layout
# plt.tight_layout()

# # Show the plots
# plt.show()


**Prescribed_meds**

In [80]:
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

# # Countplot with value counts 
# sns.countplot(x="prescribed_meds", data=data, ax=axes[0])
# axes[0].set_title("Distribution of Prescribed_Meds")
# for p in axes[0].patches:
#     axes[0].annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', xytext=(0, 1), textcoords='offset points')

# # Pie chart
# data['prescribed_meds'].value_counts().plot.pie(autopct="%.1f%%", ax=axes[1])
# axes[1].set_title("Proportion of Prescribed_meds")
# plt.tight_layout()
# plt.show()

In [81]:
# sns.catplot(x = 'prescribed_meds', y = 'readmitted_binary', data=data, kind = "bar", height= 5)
# plt.title("Readmitted Probability")
# plt.show()

The categorical features are all ready. Let's do some data visualization in the numerical features and check for outliers

In [82]:
numerical_features = ['age', 'outpatient_visits_in_previous_year', 'emergency_visits_in_previous_year', 
      'inpatient_visits_in_previous_year', 'average_pulse_bpm', 'length_of_stay_in_hospital',
      'number_lab_tests','non_lab_procedures', 'number_of_medications', 'number_diagnoses', 'num_prescribed_meds']

In [83]:
# plt.figure(figsize=(20,6))
# correlation_matrix = data[numerical_features].corr()
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
# plt.title('Correlation Heatmap')
# plt.show()

We can see some correlation between some of the variables, but nothing to worry about, atleast for now

In [84]:
# # Calculate the number of rows needed
# num_rows = math.ceil(len(data[numerical_features].columns) / 3)

# # Create subplots with the calculated number of rows and 3 columns
# fig, axes = plt.subplots(nrows=num_rows, ncols=3, figsize=(20, 5 * num_rows))

# # Flatten the axes array to simplify indexing
# axes = axes.flatten()

# # Iterate over the columns and create histograms
# for i, column in enumerate(numerical_features):
#     sns.histplot(data[column], bins=30, kde=True, ax=axes[i])
#     axes[i].set_title(f'Histogram for {column}')

# # Adjust layout
# plt.tight_layout()

# # Show the plots
# plt.show()


In [85]:
# # Calculate the number of rows needed
# num_rows = math.ceil(len(data[numerical_features].columns) / 3)

# # Create subplots with the calculated number of rows and 3 columns
# fig, axes = plt.subplots(nrows=num_rows, ncols=3, figsize=(20, 5 * num_rows))

# # Flatten the axes array to simplify indexing
# axes = axes.flatten()

# # Iterate over the columns and create box plots by class
# for i, column in enumerate(numerical_features):  # Exclude the target variable
#     sns.boxplot(x='readmitted_binary', y=column, data=data, ax=axes[i])
#     axes[i].set_title(f'{column}')

# # Adjust layout
# plt.tight_layout()

# # Show the plots
# plt.show()


As we can see there are outliers in most of the variables. Let's try to get rid of some outliers by applying winsonorizing

In [86]:
from scipy.stats.mstats import winsorize

for col in numerical_features:
    data[col] = winsorize(data[col], limits=[0.01, 0.01])

In [87]:
# # Calculate the number of rows needed
# num_rows = math.ceil(len(data[numerical_features].columns) / 3)

# # Create subplots with the calculated number of rows and 3 columns
# fig, axes = plt.subplots(nrows=num_rows, ncols=3, figsize=(20, 5 * num_rows))

# # Flatten the axes array to simplify indexing
# axes = axes.flatten()

# # Iterate over the columns and create histograms
# for i, column in enumerate(numerical_features):
#     sns.histplot(data[column], bins=30, kde=True, ax=axes[i])
#     axes[i].set_title(f'Histogram for {column}')

# # Adjust layout
# plt.tight_layout()

# # Show the plots
# plt.show()


In [88]:
# # Calculate the number of rows needed
# num_rows = math.ceil(len(data[numerical_features].columns) / 3)

# # Create subplots with the calculated number of rows and 3 columns
# fig, axes = plt.subplots(nrows=num_rows, ncols=3, figsize=(20, 5 * num_rows))

# # Flatten the axes array to simplify indexing
# axes = axes.flatten()

# # Iterate over the columns and create box plots by class
# for i, column in enumerate(numerical_features):  # Exclude the target variable
#     sns.boxplot(x='readmitted_binary', y=column, data=data, ax=axes[i])
#     axes[i].set_title(f'{column}')

# # Adjust layout
# plt.tight_layout()

# # Show the plots
# plt.show()


Much better. We have reduced the number of outliers by a lot. For the rest of the outliers we decided to keep them for now. We might consider binning for some of these numerical variables as some of them have high skewness, but for now we will go like this

**Feature Engineering**

We will create the following variables: 
* **patient_dup**  - if the patient is duplicated
* **num_patient_dup** - number of times the patient is duplicated
* **service_utilization_in_previous_year** - sum of outpatient, emergency and inpatient visits in previous years
* **num_readmissions** - number of readmissions
* **num_readmissions_in30** - number of readmissions in 30 days
* **num_readmissions_over30** - number of readmissions over 30 days

Save index to use later

In [89]:
data_index = data.index

In [90]:
data['patient_dup'] = data['patient_id'].duplicated(keep=False).astype(int)

In [91]:
data['num_patient_dup'] = data.groupby('patient_id')['patient_id'].transform('count') - 1

In [92]:
data['service_utilization_in_previous_year'] =  data['outpatient_visits_in_previous_year'] + \
                                                data['emergency_visits_in_previous_year'] + data['inpatient_visits_in_previous_year']

In [93]:
count_df = data[data['readmitted_multiclass'] != 'No'].groupby('patient_id').size().reset_index(name='num_readmissions')

data = pd.merge(data, count_df, on='patient_id', how='left')
data['num_readmissions'].fillna(0, inplace=True)

In [94]:
count_df = data[data['readmitted_multiclass'] == '>30 days'].groupby('patient_id').size().reset_index(name='num_readmissions_over30')

data = pd.merge(data, count_df, on='patient_id', how='left')
data['num_readmissions_over30'].fillna(0, inplace=True)

In [95]:
count_df = data[data['readmitted_multiclass'] == '<30 days'].groupby('patient_id').size().reset_index(name='num_readmissions_in30')

data = pd.merge(data, count_df, on='patient_id', how='left')
data['num_readmissions_in30'].fillna(0, inplace=True)

In [96]:
data = data.set_index(data_index)
#data.head()

**Modeling**

Let's prepare the variables for modeling, we will start by spliiting the target variable from the rest. For now we will not use the following variable **num_readmissions** to prevent data leakage, but we will do some tests with it later

In [97]:
X = data.drop(columns=['readmitted_binary', 'readmitted_multiclass', 'patient_id', 'num_readmissions', 'num_readmissions_in30', 'num_readmissions_over30'])
y = data['readmitted_binary']

Now we will use label encoder for the categorical variables and power transformer for the numerical variables

In [98]:
numerical_features.extend(['num_patient_dup', 'service_utilization_in_previous_year'])

In [99]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

categorical_features = X.select_dtypes(include='object').columns

# Create a dictionary to store label encoders
label_encoders = {}

# Apply label encoding to the training data
for feature in categorical_features:
    le = LabelEncoder()
    X[feature] = le.fit_transform(X[feature].astype(str))
    label_encoders[feature] = le

In [100]:
from sklearn.preprocessing import PowerTransformer

pt = PowerTransformer()
X[numerical_features] = pt.fit_transform(X[numerical_features])

Now let's start with the modeling. First we will create a cross validation function to use in the models. We will use undersampling to balance the dataset. We tried other techniques but this was the best one. We also tried with differente sampling strategies and 1/1.6 was the best one

In [101]:
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import f1_score
from statistics import mean

def cross_validation(model, X, y, sampler):
    scoring_list = []

    # Define the cross-validation strategy (StratifiedKFold is used here)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Perform cross-validation with undersampling
    for train_index, test_index in cv.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply undersampling to the training set
        X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)

        # Train your model on the resampled training set
        model.fit(X_resampled, y_resampled)

        # Make predictions on the test set
        predictions = model.predict(X_test)

        # Evaluate the model
        f1 = f1_score(y_test, predictions)
        scoring_list.append(f1)
        
    model_name = type(model).__name__
    print(f'{model_name} CV score:', mean(scoring_list))

Let's test some models with our function

In [102]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
# from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.ensemble import HistGradientBoostingClassifier
# from imblearn.over_sampling import SMOTE

# models = [
#     LogisticRegression(random_state=42),
#     RandomForestClassifier(random_state=42),
#     GradientBoostingClassifier(random_state=42),
#     SVC(random_state=42, class_weight='balanced'),
#     KNeighborsClassifier(),
#     GaussianNB(),
#     DecisionTreeClassifier(random_state=42),
#     AdaBoostClassifier(random_state=42),
#     MLPClassifier(random_state=42),
#     HistGradientBoostingClassifier(random_state=42)
# ]

# rus = RandomUnderSampler(sampling_strategy=1/1.6, random_state=42)

# print('RUS')
# for model in models:
#     cross_validation(model, X, y, rus)

As we can see our best model is the GradientBoostingClassifier. We will use GridSearch on it to do some hyper parameter tuning 

In [103]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import make_scorer

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# #Set the parameters we want to tune and the values we wants to test
# param_grid = {
#     'learning_rate': [0.01, 0.1, 0.2],
#     'n_estimators': [50, 100, 200],
#     'max_depth': [3, 5, 7],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# scorer = make_scorer(f1_score)

# undersample = RandomUnderSampler(sampling_strategy=1/1.6, random_state=42)
# X_train_balanced, y_train_balanced = undersample.fit_resample(X_train, y_train)

# gs = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=3, scoring=scorer, verbose=1, n_jobs=-1)
# gs.fit(X_train_balanced, y_train_balanced)

# test_f1_score = f1_score(y_test, gs.predict(X_test))
# print('F1 score on training set:', gs.best_score_)
# print('F1 score on the test set:', test_f1_score)
# print("Best Parameters:", gs.best_params_)

As we can see Hyperparameter Tuning didn't help, so we will just use the default model

In [104]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

rus = RandomUnderSampler(sampling_strategy=1/1.6, random_state=42)

gbc = GradientBoostingClassifier(random_state=42)
cross_validation(gbc, X, y, rus)

GradientBoostingClassifier CV score: 0.3565081786453042


**Pipeline**

Let's create a pipeline to prepare the test data for modeling

The code commented next was used for kaggle competetion but we figured it would be better to be commented to be less time consuming for the teachers

In [105]:
# def initial_preprocessing(data):
#     data = data.set_index('encounter_id')
#     data.drop(['country'], inplace=True, axis=1)
#     categorical_columns = data.select_dtypes(include=['object']).columns
#     data[categorical_columns] = data[categorical_columns].apply(lambda x: x.str.lstrip())
#     data = data.replace(['Not Available', 'Not Mapped', '?', 'Unknown/Invalid'], np.nan)
#     return data

In [106]:
# def handle_missing_values(data):
#     data['race'].fillna('Unknown', inplace=True)
#     data['gender'].fillna(data['gender'].mode()[0], inplace=True)

#     # Define a mapping from age groups to numeric values
#     age_mapping = {
#     '[0-10)': 5,
#     '[10-20)': 15,
#     '[20-30)': 25,
#     '[30-40)': 35,
#     '[40-50)': 45,
#     '[50-60)': 55,
#     '[60-70)': 65,
#     '[70-80)': 75,
#     '[80-90)': 85,
#     '[90-100)': 95
#     }

#     # map values to numerical values
#     data['age'] = data['age'].replace(age_mapping)
#     data['age'] = data['age'].fillna(data['age'].median())

#     data.drop(columns=['weight'], inplace=True)
#     data['payer_code'].fillna('Unknown', inplace=True)
#     data['glucose_test_result'].fillna('Not Taken', inplace=True)
#     data['a1c_test_result'].fillna('Not Taken', inplace=True)
#     data['admission_type'].fillna('Unknown', inplace=True)
#     data['medical_specialty'].fillna('Unknown', inplace=True)
#     data['discharge_disposition'].fillna('Unknown', inplace=True)
#     data['admission_source'].fillna('Unknown', inplace=True)
#     data['primary_diagnosis'].fillna('Unknown', inplace=True)
#     data['secondary_diagnosis'].fillna('Unknown', inplace=True)
#     data['additional_diagnosis'].fillna('Unknown', inplace=True)
#     return data

In [107]:
# def handle_high_cardinality(data):
#     data['has_insurance'] = data['payer_code'].map(lambda x: 'No' if x == 'Unknown' else 'Yes')
#     data.drop(columns=['payer_code'], inplace=True)

#     mapped_admission_type = {"Trauma Center":"Emergency","Newborn":"Urgent"}
#     data['admission_type'] = data['admission_type'].replace(mapped_admission_type)

#     data.loc[data['discharge_disposition'].str.contains("expired", case=False, na=False), 'discharge_disposition'] =  "Expired"

#     admission_source_mapping = {
#         'Clinic Referral': 'Referral',
#         'Physician Referral': 'Referral',
#         'HMO Referral': 'Referral',
#         'Transfer from another health care facility': 'Transfer',
#         'Transfer from a hospital': 'Transfer',
#         'Transfer from a Skilled Nursing Facility (SNF)': 'Transfer',
#         'Transfer from hospital inpt/same fac reslt in a sep claim': 'Transfer',
#         'Transfer from critial access hospital': 'Transfer',
#         'Transfer from Ambulatory Surgery Center': 'Transfer',
#         'Court/Law Enforcement': 'Other',
#         'Extramural Birth': 'Other',
#         'Normal Delivery': 'Other',
#         'Sick Baby': 'Other',
#         'Unknown': 'Other'
#     }

#     data['admission_source'] = data['admission_source'].replace(admission_source_mapping)

#     icd9_mapping = {
#         '001-139': 'Infectious and parasitic diseases',
#         '140-239': 'Neoplasms',
#         '240-279': 'Endocrine, nutritional and metabolic diseases, and immunity disorders',
#         '280-289': 'Diseases of the blood and blood-forming organs',
#         '290-319': 'Mental disorders',
#         '320-389': 'Diseases of the nervous system and sense organs',
#         '390-459': 'Diseases of the circulatory system',
#         '460-519': 'Diseases of the respiratory system',
#         '520-579': 'Diseases of the digestive system',
#         '580-629': 'Diseases of the genitourinary system',
#         '630-679': 'Complications of pregnancy, childbirth, and the puerperium',
#         '680-709': 'Diseases of the skin and subcutaneous tissue',
#         '710-739': 'Diseases of the musculoskeletal system and connective tissue',
#         '740-759': 'Congenital anomalies',
#         '760-779': 'Certain conditions originating in the perinatal period',
#         '780-799': 'Symptoms, signs, and ill-defined conditions',
#         '800-999': 'Injury and poisoning',
#     }


#     def is_numeric(input_string):
#         try:
#             float(input_string)  # Try to convert the string to a float
#             return True
#         except ValueError:
#             return False


#     def map_icd9(value):
#         for key in icd9_mapping:
#             code_range = key.split('-')
#             if is_numeric(value):   
#                 if int(code_range[0]) <= float(value) <= int(code_range[1]):
#                     return icd9_mapping[key]
#             else:
#                 if value.startswith(('E', 'V')):
#                     return 'External causes of injury and supplemental classification'
#                 else:
#                     return 'Unknown_diagnosis'


#     data['primary_diagnosis'] = data['primary_diagnosis'].apply(map_icd9)
#     data['secondary_diagnosis'] = data['secondary_diagnosis'].apply(map_icd9)
#     data['additional_diagnosis'] = data['additional_diagnosis'].apply(map_icd9)

#     glucose_mapping = {'>300': 3,
#                 '>200': 2,
#                 'Norm': 1,
#                 'Not Taken': 0}

#     data['glucose_test_result'] = data['glucose_test_result'] .replace(glucose_mapping)

#     glucose_test_mapping = {3: 'Yes',
#                 2: 'Yes',
#                 1: 'Yes',
#                 0: 'No'}

#     data['glucose_test_taken'] = data['glucose_test_result'].replace(glucose_test_mapping)

#     a1c_mapping = {'>7': 2,
#                 '>8': 3,
#                 'Norm': 1,
#                 'Not Taken': 0}

#     data['a1c_test_result'] = data['a1c_test_result'] .replace(a1c_mapping)

#     a1c_test_mapping = {3: 'Yes',
#                 2: 'Yes',
#                 1: 'Yes',
#                 0: 'No'}

#     data['a1c_test_taken'] = data['a1c_test_result'] .replace(a1c_test_mapping)



#     data['medication'] = data['medication'].apply(ast.literal_eval)

#     # Extract unique medications from the lists
#     unique_medications = set(med for meds in data['medication'] for med in meds)

#     # Create binary columns for each unique medication type
#     for med in unique_medications:
#         data[med] = data['medication'].apply(lambda x: 'Yes' if med in x else 'No')

#     data['num_prescribed_meds'] = data['medication'].apply(len)
#     data['prescribed_meds'] = data['num_prescribed_meds'].apply(lambda x: 'Yes' if x > 0 else 'No')
    
#     # drop medication column
#     data.drop(columns=['medication'], inplace=True)

#     columns_to_drop =  []

#     for med in unique_medications:
#         class_1_percentage = ((data[med].value_counts(normalize=True)*100)[1])
#         if class_1_percentage < 1:
#             columns_to_drop.append(med)

#     data.drop(columns=columns_to_drop, inplace=True)
#     return data


In [108]:
# def feature_engineering(data):
#     data['num_patient_dup'] = data.groupby('patient_id')['patient_id'].transform('count') - 1
#     data['patient_dup'] = data['patient_id'].duplicated(keep=False).astype(int)
#     data['service_utilization_in_previous_year'] =  data['outpatient_visits_in_previous_year'] + \
#                                                     data['emergency_visits_in_previous_year'] + data['inpatient_visits_in_previous_year']
#     return data

In [109]:
# #diogo - "C:/Users/diogo/Desktop/ML PROJETO/project_data (1)/test.csv"

# test = pd.read_csv("files/test.csv")

In [110]:
# test = initial_preprocessing(test)
# test = handle_missing_values(test)
# test = handle_high_cardinality(test)
# test = feature_engineering(test)

Now let's join the patient_dup and num_patient_dup columns from the train to the test

Save the test index to use later

In [111]:
# test_index = test.index

Joining patient_dup from training with test

In [112]:
# def join_patient_dup(data, test):
#     pat_mapping = data[['patient_id', 'patient_dup', 'num_patient_dup']]

#     common_patients = pat_mapping[pat_mapping['patient_id'].isin(test['patient_id'])]

#     duplicated_common_patients = common_patients[common_patients['patient_dup'] == 1]

#     test.loc[test['patient_id'].isin(duplicated_common_patients['patient_id']), 'patient_dup'] = 1
#     return test

# test = join_patient_dup(data, test)

Updating num_patient_dup in the test by summing based on the patient_id from the training set

In [113]:
# def update_num_patient_dup(data, test):
#     pat_mapping = data[['patient_id', 'patient_dup', 'num_patient_dup']]

#     common_patients = pat_mapping[pat_mapping['patient_id'].isin(test['patient_id'])]

#     duplicated_common_patients = common_patients[common_patients['patient_dup'] == 1]

#     sums_by_patient_id = duplicated_common_patients.groupby('patient_id')['num_patient_dup'].sum().reset_index()

#     # Merge num_patient_dup with test DataFrame based on 'patient_id'
#     test = pd.merge(test, sums_by_patient_id, on='patient_id', how='left', suffixes=('', '_sum'))

#     # Fill NaN values in 'num_patient_dup_sum' with 0
#     test['num_patient_dup_sum'].fillna(0, inplace=True)

#     # Update 'num_patient_dup' in test
#     test['num_patient_dup'] += test['num_patient_dup_sum'].astype(int)

#     # Drop the temporary column 'num_patient_dup_sum'
#     test.drop('num_patient_dup_sum', axis=1, inplace=True)

#     test = test.set_index(test_index)

#     return test

# test = update_num_patient_dup(data,test)

Replace problematic categories with 'Unknown' and drop 'patient_id'

In [114]:
# test_med_spec_mapping = {'Perinatology': 'Unknown',
#  'Dermatology': 'Unknown',
#  'Psychiatry-Addictive': 'Unknown',
#  'Surgery-PlasticwithinHeadandNeck': 'Unknown'}

# test['medical_specialty'] = test['medical_specialty'].replace(test_med_spec_mapping)
# test.drop(columns=['patient_id'],inplace=True)

Let's apply label encoding and the power transformer to the test data, and right after we order the columns based on the training data columns

In [115]:
# for feature in categorical_features:
#     le = label_encoders[feature]
#     test[feature] = le.transform(test[feature].astype(str))

In [116]:
# pt = PowerTransformer()
# test[numerical_features] = pt.fit_transform(test[numerical_features])

In [117]:
# ordered_columns = X.columns
# test = test[ordered_columns]

Submission

In [118]:
# test_pred = gbc.predict(test)
# submit = pd.DataFrame(test_pred, columns=['readmitted_binary'], index=test.index)
# submit['readmitted_binary'] = submit['readmitted_binary'].map({0: 'No', 1: 'Yes'})
# submit.to_csv('submit.csv')

We have got a nice result of 0.3496 in the test set. Now we will test the model with the variables we had dropped earlier to prevent data leakage

In [119]:
# X = data.drop(columns=['readmitted_binary', 'readmitted_multiclass', 'patient_id'])
# y = data['readmitted_binary']

In [120]:
# for feature in categorical_features:
#     le = label_encoders[feature]
#     X[feature] = le.transform(X[feature].astype(str))

In [121]:
# numerical_features_2 = numerical_features + ['num_readmissions', 'num_readmissions_in30', 'num_readmissions_over30']

# pt = PowerTransformer()
# X[numerical_features_2] = pt.fit_transform(X[numerical_features_2])

Test only the best models we got earlier to be more time efficient

In [122]:
# models = [
#     RandomForestClassifier(random_state=42),
#     GradientBoostingClassifier(random_state=42),
#     AdaBoostClassifier(random_state=42),
#     HistGradientBoostingClassifier(random_state=42)
# ]


# rus = RandomUnderSampler(sampling_strategy=1/1.6, random_state=42)

# for model in models:
#     cross_validation(model, X, y, rus)

In [123]:
# from sklearn.metrics import classification_report
# from sklearn.model_selection import train_test_split

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# gb_classifier = HistGradientBoostingClassifier(random_state=42)

# undersample = RandomUnderSampler(sampling_strategy=1/1.6) 
# X_train_balanced, y_train_balanced = undersample.fit_resample(X_train, y_train)

# # Fit the model to the training data
# gb_classifier.fit(X_train_balanced, y_train_balanced)

# # Make predictions on the test set
# y_pred = gb_classifier.predict(X_test)

# print('Classification Report:')
# print(classification_report(y_test, y_pred))

Let's create these variables in the test set

In [124]:
# test = pd.read_csv("files/test.csv")
# test = initial_preprocessing(test)
# test = handle_missing_values(test)
# test = handle_high_cardinality(test)
# test = feature_engineering(test)

# test_index = test.index
# test = join_patient_dup(data, test)
# test = update_num_patient_dup(data,test)
# test['medical_specialty'] = test['medical_specialty'].replace(test_med_spec_mapping)

# for feature in categorical_features:
#     le = label_encoders[feature]
#     test[feature] = le.transform(test[feature].astype(str))


In [125]:
# common_patients = data[data['patient_id'].isin(test['patient_id'])]
# patient_id_to_readmissions = dict(zip(common_patients['patient_id'], common_patients['num_readmissions']))
# test['num_readmissions'] = test['patient_id'].map(patient_id_to_readmissions)
# test = test.fillna(0)

In [126]:
# common_patients = data[data['patient_id'].isin(test['patient_id'])]
# patient_id_to_readmissions = dict(zip(common_patients['patient_id'], common_patients['num_readmissions_in30']))
# test['num_readmissions_in30'] = test['patient_id'].map(patient_id_to_readmissions)
# test = test.fillna(0)

In [127]:
# common_patients = data[data['patient_id'].isin(test['patient_id'])]
# patient_id_to_readmissions = dict(zip(common_patients['patient_id'], common_patients['num_readmissions_over30']))
# test['num_readmissions_over30'] = test['patient_id'].map(patient_id_to_readmissions)
# test = test.fillna(0)

In [128]:
# test.drop(columns=['patient_id'], inplace=True)

In [129]:
# pt = PowerTransformer()
# test[numerical_features_2] = pt.fit_transform(test[numerical_features_2])

In [130]:
# ordered_columns = X.columns
# test = test[ordered_columns]

In [131]:
# test_pred = gb_classifier.predict(test)
# submit = pd.DataFrame(test_pred, columns=['readmitted_binary'], index=test.index)
# submit['readmitted_binary'] = submit['readmitted_binary'].map({0: 'No', 1: 'Yes'})
# submit.to_csv('submit.csv')

We got very good scores on the training and validations sets when we tried the new variables, but in the test sets the results were a lot worse than what we had. This is probably due to the data leakage

In [132]:
# categorical_features

**Multiclass**

In [133]:
data2=data.copy()
numerical_features2 = numerical_features
categorical_features2 = categorical_features

In [134]:
# data2.head()

In [135]:
data2['readmitted_multiclass'] = data2['readmitted_multiclass'].replace({'No': 0, '>30 days': 1, '<30 days': 2})

### Aditional preprocessing in multiclass

In [136]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Replace 'your_dataframe' with your DataFrame variable
# # and 'target_column' with your target column name.

# # Identifying categorical columns (excluding the target column)
# categorical_columns = data2.select_dtypes(include=['object', 'category']).columns
# categorical_columns = [col for col in categorical_columns if col != 'readmitted_multiclass']

# # Creating plots for each categorical feature
# for column in categorical_columns:
#     # Counting occurrences and normalizing to get probabilities
#     count_df = data2.groupby([column, 'readmitted_multiclass']).size().unstack(fill_value=0)
#     prob_df = count_df.div(count_df.sum(axis=1), axis=0)

#     # Plotting
#     prob_df.plot(kind='bar', stacked=True, figsize=(10,6))
#     plt.xlabel(column.capitalize())
#     plt.ylabel('Probability')
#     plt.title(f'Probability of Target Variable for Each {column.capitalize()}')
#     plt.legend(title='Target', labels=['0', '1', '2'])
#     plt.show()

 as we can see, a lot of features dont show a relevcancy for thr predictions because the target variable distribution is roughly the same across different groups of the feature, it may suggest that some features dont have a strong predictive power for the target variable

In [137]:
# # and 'target_column' with your target column name.
# import pandas as pd
# from scipy.stats import chi2_contingency
# df = data2
# target_column = 'readmitted_multiclass'

# # List to keep track of p-values
# chi_squared_results = []

# # Iterating over each categorical feature
# for column in df.select_dtypes(include=['object', 'category']).columns:
#     if column != target_column:
#         # Creating a contingency table
#         contingency_table = pd.crosstab(df[column], df[target_column])

#         # Chi-squared test
#         chi2, p, dof, ex = chi2_contingency(contingency_table, correction=False)
        
#         # Append the result to the list
#         chi_squared_results.append((column, chi2, p))

# # Creating a DataFrame to display results
# chi_squared_df = pd.DataFrame(chi_squared_results, columns=['Feature', 'Chi2 Statistic', 'P-value'])

# # Sorting by p-value
# chi_squared_df = chi_squared_df.sort_values(by='P-value')

# print(chi_squared_df)

In [138]:
## we can drop the ones with higher than 0.05 p-value bc they are not statistically significant

In [139]:
data2.drop(columns = ['glyburide'],inplace=True)


In [140]:
categorical_features2 = [col for col in categorical_features2 if col not in ['glyburide']]

In [141]:
# correlation_matrix = data2.corr()

# # Select correlations of the target variable with other features
# target_correlation = correlation_matrix['readmitted_multiclass']

# # If you want to drop the target variable's self-correlation
# target_correlation = target_correlation.drop(labels=['readmitted_multiclass'])
# print("\nCorrelation of Features with the Target Variable (excluding self-correlation):")
# print(target_correlation)

maybe drop average_pulse_bpm bc its very very low related

In [142]:
data2.drop(columns = ['average_pulse_bpm'],inplace=True)


In [143]:
numerical_features2 = [col for col in numerical_features2 if col not in ['average_pulse_bpm']]

In [144]:

# # Set the target variable
# target_variable = 'readmitted_multiclass'

# # Loop through each column and create a violin plot
# for column in data2[numerical_features2].columns:
#     plt.figure(figsize=(10, 6))
#     sns.violinplot(x=target_variable, y=column, data=df)
#     plt.title(f'Violin Plot of {column} by {target_variable}')
#     plt.xlabel(target_variable)
#     plt.ylabel(column)
#     plt.show()

it didnt do anything good

In [145]:
# # Variable for which you want to calculate the percentages
# variable = 'num_patient_dup'  # Replace with the actual name of your variable

# # Loop through each unique value in the variable
# for unique_value in data2[variable].unique():
#     # Filter the DataFrame for the current unique value
#     filtered_df = data2[data2[variable] == unique_value]

#     # Calculate the percentage of 0, 1, and 2 in the target variable for this subset
#     value_counts = filtered_df['readmitted_multiclass'].value_counts(normalize=True) * 100  # Replace 'Target' with your target variable name

#     # Print the percentages
#     print(f'For {variable} = {unique_value}:')
#     for value in [0, 1, 2]:
#         percentage = value_counts.get(value, 0)  # Get the percentage for each value, defaulting to 0 if not found
#         print(f'  Percentage of {value} in Target: {percentage:.2f}%')
#     print()

didnt take anything from here

In [146]:
# for column in data2[numerical_features2].columns:
#     plt.figure(figsize=(10, 4))
#     sns.histplot(data2[column], kde=True)
#     plt.title(f'Distribution of {column}')
#     plt.xlabel(column)
#     plt.ylabel('Frequency')
#     plt.show()

applied box-cox transformations for the skewed distributed and with no 0's

In [147]:
import numpy as np
import pandas as pd
from scipy import stats


# Columns to transform
columns_to_transform = ['length_of_stay_in_hospital', 'number_of_medications']  # Replace with actual column names

# Loop through each column and apply Box-Cox transformation
for column in columns_to_transform:
    # Check if the column contains zero or negative values
    if data2[column].min() <= 0:
        # Shift the column values to be strictly positive
        data2[column] += (1 - data2[column].min())

    # Apply Box-Cox transformation
    data2[f'boxcox_{column}'], _ = stats.boxcox(data2[column])

# Display the first few rows of the DataFrame to verify the transformation
print(data2[[*columns_to_transform, *[f'boxcox_{col}' for col in columns_to_transform]]].head())

              length_of_stay_in_hospital  number_of_medications  \
encounter_id                                                      
533253                                 2                     20   
426224                                14                     25   
634063                                 6                     22   
890610                                 6                      9   
654194                                 6                     15   

              boxcox_length_of_stay_in_hospital  boxcox_number_of_medications  
encounter_id                                                                   
533253                                 0.725748                      5.037858  
426224                                 3.155394                      5.643808  
634063                                 2.020663                      5.291367  
890610                                 2.020663                      3.193172  
654194                                 2.020663   

In [148]:
numerical_features2.extend(['boxcox_length_of_stay_in_hospital',  'boxcox_number_of_medications'])

In [149]:
numerical_features2 = [col for col in numerical_features2 if col not in ['length_of_stay_in_hospital','number_of_medications']]

In [150]:
# for column in ['boxcox_length_of_stay_in_hospital',  'boxcox_number_of_medications']:
#     plt.figure(figsize=(10, 4))
#     sns.histplot(data2[column], kde=True)
#     plt.title(f'Distribution of {column}')
#     plt.xlabel(column)
#     plt.ylabel('Frequency')
#     plt.show()

In [151]:
X = data2.drop(columns=['readmitted_binary', 'readmitted_multiclass', 'patient_id', 'num_readmissions', 'num_readmissions_in30', 'num_readmissions_over30'])
y = data2['readmitted_multiclass']

In [152]:
for feature in categorical_features2:
    le = LabelEncoder()
    X[feature] = le.fit_transform(X[feature].astype(str))
    label_encoders[feature] = le


pt = PowerTransformer()
X[numerical_features2] = pt.fit_transform(X[numerical_features2])

In [153]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Instantiate the RandomUnderSampler
undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)



### RFE CV 
- since running rfecv takes a lot of time we just created a variable with the features that were choosen by the algorithm

In [154]:
# estimator = GradientBoostingClassifier()  # or another classifier

# from sklearn.feature_selection import RFECV
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import StratifiedKFold
# # Create the RFECV object, using a stratified k-fold cross-validation
# rfecv = RFECV(estimator, stp=1, cv=StratifiedKFold(5), scoring='accuracy')

# # Fit RFECV to the training data
# rfecv.fit(X_train, y_train)

# # The mask of selected features can be obtained with `rfecv.support_`
# selected_features = X_train.columns[rfecv.support_]

# # Print the number of features selected and their names
# print(f"Optimal number of features: {rfecv.n_features_}")
# print(f"Selected features: {selected_features}")

In [155]:
selected_features = ['race', 'age', 'outpatient_visits_in_previous_year',
       'emergency_visits_in_previous_year',
       'inpatient_visits_in_previous_year', 'admission_type',
       'medical_specialty', 'discharge_disposition', 'admission_source',
       'length_of_stay_in_hospital', 'number_lab_tests',
       'number_of_medications', 'primary_diagnosis', 'secondary_diagnosis',
       'additional_diagnosis', 'number_diagnoses', 'has_insurance',
       'glucose_test_taken', 'num_prescribed_meds', 'patient_dup',
       'num_patient_dup', 'service_utilization_in_previous_year',
       'boxcox_number_of_medications']

In [156]:
X_train = X_train[selected_features]
X_test = X_test[selected_features]

In [157]:
 #Instantiate the RandomUnderSampler
undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)

# Fit and transform the training data
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Replace RandomForestClassifier with your chosen model
model = GradientBoostingClassifier()

# Train the model on the resampled training set
model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Print classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.80      0.78      7681
           1       0.57      0.35      0.43      4977
           2       0.25      0.51      0.34      1590

    accuracy                           0.61     14248
   macro avg       0.53      0.55      0.52     14248
weighted avg       0.64      0.61      0.61     14248



We got a accuracy score of 0.61, which is acceptable. We could have gotten a higher accuracy score if we didn't balance the classes but that would just assing the predictions to the majority class, and that wouldn't be considered a useful model. So we focused having a good recall, precision and f1 score in the minority classes. Now we will do cross validation to validate our results

##### the next one is without undersampling

In [158]:
# # Replace RandomForestClassifier with your chosen model
# model = GradientBoostingClassifier()

# # Train the model on the resampled training set
# model.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = model.predict(X_test)

# # Print classification report
# print(classification_report(y_test, y_pred))

In [159]:
# from sklearn.model_selection import StratifiedKFold
# from imblearn.under_sampling import RandomUnderSampler
# from sklearn.metrics import accuracy_score
# from statistics import mean

# def cross_validation_mc(model, X, y, sampler):
#     scoring_list = []

#     # Define the cross-validation strategy (StratifiedKFold is used here)
#     cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#     # Perform cross-validation with undersampling
#     for train_index, test_index in cv.split(X, y):
#         X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#         y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#         # Apply undersampling to the training set
#         X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)

#         # Train your model on the resampled training set
#         model.fit(X_resampled, y_resampled)

#         # Make predictions on the test set
#         predictions = model.predict(X_test)

#         # Evaluate the model using accuracy
#         accuracy = accuracy_score(y_test, predictions)
#         scoring_list.append(accuracy)

#     model_name = type(model).__name__
#     print(f'{model_name} CV accuracy:', mean(scoring_list))


In [160]:
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
# from sklearn.ensemble import HistGradientBoostingClassifier

# models = [
#     RandomForestClassifier(random_state=42),
#     GradientBoostingClassifier(random_state=42),
#     AdaBoostClassifier(random_state=42),
#     HistGradientBoostingClassifier(random_state=42)
# ]

# rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)

# print('RUS')
# for model in models:
#     cross_validation_mc(model, X, y, rus)